In [2]:
import tkinter as tk
from tkinter import ttk
import json
from datetime import datetime

class BudgetTracker:
    def __init__(self):
        self.transactions = []
        self.load_from_file("transactions.json") 

    def add_transaction(self, category, amount, is_income=False):
        transaction = {
            "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "category": category,
            "amount": amount,
            "is_income": is_income
        }
        self.transactions.append(transaction)

    def calculate_balance(self):
        total_income = sum(transaction["amount"] for transaction in self.transactions if transaction["is_income"])
        total_expenses = sum(transaction["amount"] for transaction in self.transactions if not transaction["is_income"])
        return total_income - total_expenses

    def analyze_expenses(self):
        expense_categories = {}
        for transaction in self.transactions:
            if not transaction["is_income"]:
                category = transaction["category"]
                amount = transaction["amount"]
                if category in expense_categories:
                    expense_categories[category] += amount
                else:
                    expense_categories[category] = amount
        return expense_categories

    def save_to_file(self, filename):
        with open(filename, "w") as file:
            for transaction in self.transactions:
                file.write(json.dumps(transaction) + '\n')

    def load_from_file(self, filename):
        try:
            with open(filename, "r") as file:
                self.transactions = [json.loads(line.strip()) for line in file]
        except FileNotFoundError:
            print("No transactions file found. Starting with an empty list.")
            self.transactions = []

class BudgetTrackerApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Budget Tracker")
        
        self.budget_tracker = BudgetTracker()

        self.main_frame = ttk.Frame(master)
        self.main_frame.pack(padx=10, pady=10)

        self.setup_widgets()

    def setup_widgets(self):
        ttk.Label(self.main_frame, text="Budget Tracker", font=("Arial", 16, "bold")).grid(row=0, column=0, columnspan=2, pady=10)

        ttk.Button(self.main_frame, text="Add Income", command=self.add_income).grid(row=1, column=0, padx=5, pady=5, sticky="ew")
        ttk.Button(self.main_frame, text="Add Expense", command=self.add_expense).grid(row=1, column=1, padx=5, pady=5, sticky="ew")
        ttk.Button(self.main_frame, text="View Remaining Budget", command=self.view_balance).grid(row=2, column=0, columnspan=2, padx=5, pady=5, sticky="ew")
        ttk.Button(self.main_frame, text="Analyze Expenses", command=self.analyze_expenses).grid(row=3, column=0, columnspan=2, padx=5, pady=5, sticky="ew")
        ttk.Button(self.main_frame, text="Save and Exit", command=self.save_and_exit).grid(row=4, column=0, columnspan=2, padx=5, pady=5, sticky="ew")

    def add_income(self):
        income_window = tk.Toplevel(self.master)
        income_window.title("Add Income")
        income_window.grab_set()

        income_category_label = ttk.Label(income_window, text="Category:")
        income_category_label.grid(row=0, column=0, padx=5, pady=5)
        income_category_entry = ttk.Entry(income_window)
        income_category_entry.grid(row=0, column=1, padx=5, pady=5)

        income_amount_label = ttk.Label(income_window, text="Amount:")
        income_amount_label.grid(row=1, column=0, padx=5, pady=5)
        income_amount_entry = ttk.Entry(income_window)
        income_amount_entry.grid(row=1, column=1, padx=5, pady=5)

        def add_income_transaction():
            category = income_category_entry.get()
            amount = float(income_amount_entry.get())
            self.budget_tracker.add_transaction(category, amount, is_income=True)
            income_window.destroy()

        ttk.Button(income_window, text="Add", command=add_income_transaction).grid(row=2, column=0, columnspan=2, padx=5, pady=5, sticky="ew")

    def add_expense(self):
        expense_window = tk.Toplevel(self.master)
        expense_window.title("Add Expense")
        expense_window.grab_set()

        expense_category_label = ttk.Label(expense_window, text="Category:")
        expense_category_label.grid(row=0, column=0, padx=5, pady=5)
        expense_category_entry = ttk.Entry(expense_window)
        expense_category_entry.grid(row=0, column=1, padx=5, pady=5)

        expense_amount_label = ttk.Label(expense_window, text="Amount:")
        expense_amount_label.grid(row=1, column=0, padx=5, pady=5)
        expense_amount_entry = ttk.Entry(expense_window)
        expense_amount_entry.grid(row=1, column=1, padx=5, pady=5)

        def add_expense_transaction():
            category = expense_category_entry.get()
            amount = float(expense_amount_entry.get())
            self.budget_tracker.add_transaction(category, amount)
            expense_window.destroy()

        ttk.Button(expense_window, text="Add", command=add_expense_transaction).grid(row=2, column=0, columnspan=2, padx=5, pady=5, sticky="ew")

    def view_balance(self):
        balance_window = tk.Toplevel(self.master)
        balance_window.title("Remaining Budget")
        balance_window.grab_set()

        remaining_budget = self.budget_tracker.calculate_balance()
        ttk.Label(balance_window, text=f"Remaining Budget: ${remaining_budget:.2f}").pack(padx=10, pady=10)

    def analyze_expenses(self):
        analysis_window = tk.Toplevel(self.master)
        analysis_window.title("Expense Analysis")
        analysis_window.grab_set()

        expense_categories = self.budget_tracker.analyze_expenses()
        for category, amount in expense_categories.items():
            ttk.Label(analysis_window, text=f"{category}: ${amount:.2f}").pack(padx=5, pady=5)

    def save_and_exit(self):
        self.budget_tracker.save_to_file("transactions.json")
        self.master.destroy()

def main():
    root = tk.Tk()
    app = BudgetTrackerApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()